In [1]:
import pandas as pd
import numpy as np
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.hybrid import HIVECOTEV1
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.classification.kernel_based import ROCKETClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate

In [2]:
def load_data():
    path = './'
    data = []
    
    total_len = 0
    
    
    for i in range(1, 19):
        if i < 10:
            file_name = f'experiment_0{i}.csv'
        
        else:
            file_name = f'experiment_{i}.csv'
            
        file = path + file_name
        sample = pd.read_csv(file)
        data.append(sample)
        total_len += len(sample)
        
        
    return data


data = load_data()

In [3]:
long_len = 0

for sample in data:
    if long_len < len(sample):
        long_len = len(sample)

# print(long_len)

new_data = []
cols = []


In [4]:
for sample in data:
    diff = long_len - len(sample)
    sample.drop(['Machining_Process'], axis=1, inplace=True)
    cols = sample.columns
    n_sample = np.array(sample)
    re_sample = np.empty((long_len, 47))
    # re_sample.fill(np.nan)
    re_sample.fill(0)
    re_sample[:len(sample), :] = n_sample
    new_data.append(re_sample)
    
new_data = np.reshape(new_data, (-1, long_len, 47))

In [5]:
# worn / unworn 기준
label = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


# 시각 검사 기준
label2 = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1]

train_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]
test_index = [11, 15, 16, 17]


train_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
test_index = [11, 13, 14, 15, 16, 17]


In [ ]:
# CNN 적용 시

# x_train = []
# x_test = []

y_train = []
y_test = []

for i in train_index:
#     x_train.append(new_data[i])
    y_train.append(label[i])
    
for i in test_index:
#     x_test.append(new_data[i])
    y_test.append(label[i])


In [ ]:
# x_test = np.reshape(x_test, (-1, long_len, 47))
# x_train = np.reshape(x_train, (-1, long_len, 47))

In [ ]:
# SKTIME 적용 시
skdata = np.empty((len(new_data), 47))
# skdata.fill(np.nan)
skdata.fill(0)
skdata = pd.DataFrame(skdata, columns=cols)

In [ ]:
for index, sample in enumerate(new_data):
    for i in range(0, 47):
        # value = list(sample[:, i])
        value = pd.Series(sample[:, i])
        # value = value.transpose()
        skdata = skdata.astype('object')
        skdata.iat[index, i] = value

In [ ]:
x_train = skdata.iloc[train_index, :]
x_test = skdata.iloc[test_index, :]

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:

# steps = [
#     ('concatenate', ColumnConcatenator),
#     ('classify', HIVECOTEV1()),
#     ]


In [ ]:
# trans = ColumnConcatenator()

In [ ]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100)),
]


In [ ]:
# trans = MiniRocketMultivariate()
# x_train = trans.fit_transform(x_train)
# x_test = trans.fit_transform(x_test)

In [ ]:

# rclf = ROCKETClassifier(random_state=1002)
# rclf.fit(x_train, y_train)
# y_pred = rclf.predict(x_test)
# score = rclf.score(x_test, y_test)

In [ ]:
# sclf = ShapeletTransformClassifier(random_state=1002)
trans = ColumnConcatenator()
x_train = trans.fit_transform(x_train)
x_test = trans.transform(x_test)

In [ ]:
# sclf.fit(x_train, y_train)
# y_pred = sclf.predict(x_test)
# score = sclf.score(x_test, y_test)

In [ ]:
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

In [ ]:
hive = HIVECOTEV1(random_state=1002)
hive.fit(x_train, y_train)
y_pred = hive.predict(x_test)
score = hive.score(x_test, y_test)